# Clasificador de emails de Thunderbird a .csv

## Objetivo
Leer los mensajes diarios de `yum update` de las máquinas, y en base a eso crear una lista en un archivo .csv, con un "ok" si no ha habido problemas con la actualización o una x en caso de haberlos o no recibir correo.

## Organización
El programa va a tener dos partes claramente diferenciadas:
1. Análisis de los mensajes
2. Escritura del resultado en un archivo .csv

Para más información, leer el README

## 0. Preparación

Para la tarea que tenemos entre manos no es necesario ningún módulo. Pero hemos de definir los objetos correspondientes al problema entre manos, a saber: número de máquinas, lista con sus nombres (ordenada), lista inicializada con "x", y mensajes de no-error.

In [18]:
import csv

In [19]:
okkeys = ["No se han seleccionado paquetes para ser actualizados","No packages marked for update", "¡Listo!", "Complete!"]
with open("/home/goznalo/CCC/servernames.csv", encoding='utf-8-sig') as file1: #modificar con el archivo pertinente
    listservers =  [server.replace('\n', '') for server in file1]
numservers = len(listservers)
okxlist = ['x'] * numservers

En la lista de máquinas, gran parte de ellas no se cuentan a la hora de revisar estos correos porque no los mandan por defecto. Esto puede deberse a que  estén de baja, que sean sondas o SAIs que no mandan mensajes, etc. Vamos a eliminarlos de la lista de "x"

In [20]:
with open("/home/goznalo/CCC/nomessages.csv", encoding='utf-8-sig') as file2: #modificar con el archivo pertinente
    nomsglist =  [False if not server.replace('\n', '') else True for server in file2]
okxlist =  [a * b for a, b in zip(okxlist,nomsglist)]

# 1. Análisis de los mensajes

Los mensajes se encuentran en un archivo (que Thunderbird considera un directorio de mensajes) dentro de un directorio local creado y usado por Thunderbird.

In [21]:
directorio = "/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/"
archivo = "prueba1" #modificar con el archivo pertinente

path = directorio + archivo

In [22]:
print(path)

/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/prueba1


Ahora abriremos el archivo (con codificación "iso-8859-1", como podemos ver al ejecutar en una shell `file -i archivo`), que contiene cada uno de los mensajes de esa carpeta, y los separaremos en elementos de una lista, para luego iterar sobre ellos. Para efectuar esta separación, hemos de darnos cuenta de que en cada archivo de mensajes del directorio, los mensajes empiezan con una línea como por ejemplo `From - Wed Jul 29 15:16:48 2020`. Por tanto usaremos como separador `From -`

In [23]:
with open(path, 'r', encoding='iso-8859-1') as file3:
    messages = file3.read().replace('\n', '')

messlist = messages.split("From -")[1:]

Ahora que tenemos los mensajes, hemos de obtener una lista con los remitentes de cada uno de ellos, a la vez que comprobamos si tienen errores o no.

In [24]:
for mess in messlist:
    sender = mess.split("<root@")[1].split(">Received:")[0]
    num = listservers.index(sender)
    #print(listservers[num])
    for oks in okkeys:
        if oks in mess:
            okxlist[num] = "ok"
            break

print(okxlist)

['x', 'x', 'x', 'x', '', 'x', 'x', '', '', '', '', '', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '', '', 'x', 'x', '', '', '', 'x', 'x', 'x', '', 'x', 'x', 'x', 'x', '', 'x', 'x', 'x', 'x', 'ok', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '', '', 'x', '', 'x', 'x', 'x', '', 'x', '', 'x', '', 'x', 'x', '', 'x', '', 'x', '', '', '', 'x', 'x', 'x', '', '', '', 'x', '', '', 'x', 'x', 'x', 'x', 'x', '', 'x', '', 'x', '', '', '', 'x', 'x', '', 'x', 'x', 'x', '', '', 'x', 'x', 'x', 'x', '', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '

## 2. Escritura del csv

Ya tenemos la lista con los datos que buscábamos. Ahora lo que hemos de hacer es convertirla en un archivo .csv, en formato columna.

In [26]:
with open('returns.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for val in okxlist:
        writer.writerow([val])

Voilà